Note: make top line #!/usr/bin/env python

In [1]:
import subprocess
import os
import pandas as pd
import numpy as np
from typing import Tuple

In [2]:
# In[2]:
cwd = os.getcwd()
print(f'Here: {cwd}')

Here: /home/paulc600/local/final_smm/StMaryMilk2023-UofC/models/hype/scripts/analysis_scripts


In [3]:
def remove_invalid_values(simulated, observed):
    valid_indices = np.where((observed != -9999) & (simulated != -9999))
    return simulated[valid_indices], observed[valid_indices]


In [4]:
def remove_nan_rows(
    array1: np.ndarray, 
    array2: np.ndarray
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Removes rows from two arrays where either array has NaN values.
    Retains the first row if it doesn't have any NaN values.
    
    Arguments:
    array1: np.ndarray:
        First input array
    array2: np.ndarray
        Second input array
    
    Returns:
    cleaned_array1: : np.ndarray
        Cleaned array1 without NaN rows
    cleaned_array2: np.ndarray
        Cleaned array2 without NaN rows
    """
    # checks for and removes any rows where either array has a value of NaN at a corresponding row 
    # including the first one
    
    mask = np.logical_and(~np.isnan(array1), ~np.isnan(array2))
    if not np.isnan(array1[0]) and not np.isnan(array2[0]):
        mask[0] = True
    cleaned_array1 = array1[mask]
    cleaned_array2 = array2[mask]
    return cleaned_array1, cleaned_array2

In [5]:
def compute_kge(simulated_array, observed_array):
    """
    Computes KGE (Kling-Gupta Efficiency) between observed and simulated values.

    Parameters:
        observed_array (numpy.ndarray): Array of observed values.
        simulated_array (numpy.ndarray): Array of simulated values.

    Returns:
        float: KGE value.
    """
    
    # Calculate Pearson correlation coefficient
    correlation_coefficient = np.corrcoef(observed_array, simulated_array)[0, 1]
    
    # Calculate standard deviation ratio
    std_observed = np.std(observed_array)
    std_simulated = np.std(simulated_array)
    std_ratio = std_simulated / std_observed
    
    # Calculate bias ratio
    mean_observed = np.mean(observed_array)
    mean_simulated = np.mean(simulated_array)
    bias_ratio = mean_simulated / mean_observed
    
    # Calculate KGE
    kge = 1 - np.sqrt((correlation_coefficient - 1)**2 + (std_ratio - 1)**2 + (bias_ratio - 1)**2)
    
    return kge

In [6]:
def compute_bias(simulated_array, observed_array):
    """
    Computes bias between observed and simulated values.

    Parameters:
        observed_array (numpy.ndarray): Array of observed values.
        simulated_array (numpy.ndarray): Array of simulated values.

    Returns:
        float: Bias value.
    """
    
    # Calculate mean bias
    mean_observed = np.mean(observed_array)
    mean_simulated = np.mean(simulated_array)
    
    # Calculate bias
    bias = mean_simulated - mean_observed
       
    # Calculate percent bias
    percent_bias = (bias / mean_observed) * 100
    
    return percent_bias

In [7]:
# Directory where Hype outputs are saved
file_path= '../../model/results/'

In [8]:
calibration_ranges = [('1981-01-01', '1984-12-31'),
               ('1990-01-01', '1998-12-31'),
               ('2004-01-01', '2007-12-31'),
               ('2013-01-01', '2015-12-31')]

In [9]:
validation_ranges = [('1985-01-01', '1989-12-31'),
               ('1999-01-01', '2003-12-31'),
               ('2008-01-01', '2012-12-31')]

In [10]:
# Create an empty list to store total KGE values for each file
calibration_kge = []

In [11]:
validation_kge= []

In [12]:
calibration_bias = []

In [13]:
validation_bias= []

In [14]:
file_names = []

In [15]:
# Iterate through files in the output directory
for filename in os.listdir(file_path):
    if filename.endswith(".txt") and filename.startswith("00"):  # Process files with prefix '00' and end with '.txt'
        filepath = os.path.join(file_path, filename)

        # Create empty lists to store observed and simulated data for each year range
        simulated_data_cal = []
        observed_data_cal = []
        simulated_data_val = []
        observed_data_val = []
        
        # Read tab-separated file into DataFrame
        df = pd.read_csv(filepath, sep='\t', index_col=0)
        df = df.iloc[1:]  # Drop the first row

        # Convert the index to datetime if it's not already in datetime format
        if not isinstance(df.index, pd.DatetimeIndex):
            df.index = pd.to_datetime(df.index)

        # Process and filter DataFrame based on calibration period
        for start_date, end_date in calibration_ranges:
            trimmed_df1 = df.loc[start_date:end_date]
            simulated_data_cal.append(trimmed_df1['cout'].values.astype(float))  # Convert to float array
            observed_data_cal.append(trimmed_df1['rout'].values.astype(float))  # Convert to float array
            
        # Process and filter DataFrame based on validation period
        for start_date, end_date in validation_ranges:
            trimmed_df2 = df.loc[start_date:end_date]
            simulated_data_val.append(trimmed_df2['cout'].values.astype(float))  # Convert to float array
            observed_data_val.append(trimmed_df2['rout'].values.astype(float))  # Convert to float array

        # Concatenate the lists of arrays into NumPy arrays
        simulated_array_cal = np.concatenate(simulated_data_cal)
        observed_array_cal = np.concatenate(observed_data_cal)
        simulated_array_val = np.concatenate(simulated_data_val)
        observed_array_val = np.concatenate(observed_data_val)
        
        # Remove invalid values (-9999) after concatenating arrays
        simulated_array_cal, observed_array_cal = remove_invalid_values(simulated_array_cal, observed_array_cal)
        simulated_array_val, observed_array_val = remove_invalid_values(simulated_array_val, observed_array_val)
        
        # check for and remove rows with nan
        simulated_array_cal, observed_array_cal= remove_nan_rows(simulated_array_cal, observed_array_cal)
        simulated_array_val, observed_array_val= remove_nan_rows(simulated_array_val, observed_array_val)
        
        # Check if both arrays have the same length
        if len(simulated_array_cal) != len(observed_array_cal):
            raise ValueError(f"Observed and simulated data arrays for file {filename} have different lengths!")
            
        # Check if both arrays have the same length
        if len(simulated_array_val) != len(observed_array_val):
            raise ValueError(f"Observed and simulated data arrays for file {filename} have different lengths!")

        # Calculate KGE and bias for the current file
        cal_kge = compute_kge(simulated_array_cal, observed_array_cal)
        
        # Save total KGE to the list
        calibration_kge.append(cal_kge)
        
        val_kge = compute_kge(simulated_array_val, observed_array_val)
        
        # Save total KGE to the list
        validation_kge.append(val_kge)
        
        # Calculate KGE and bias for the current file
        cal_bias= compute_bias(simulated_array_cal, observed_array_cal)
        
        # Save total KGE to the list
        calibration_bias.append(cal_bias)
        
                # Calculate KGE and bias for the current file
        val_bias= compute_bias(simulated_array_val, observed_array_val)
        
        # Save total KGE to the list
        validation_bias.append(val_bias)

In [16]:
gauge_names= ['Swiftcurrent Creek at Sherburne Reservoir','St. Mary River near Babb, MT',
              'St. Mary River at International Boundary', 'Milk River at Western Crossing of International Boundary',
              'North Fork Milk River above St Mary Canal near Browning','Milk River at Eastern Crossing',
              'Big Sandy Creek at Mouth','Clear Creek at Mouth','Lodge Creek at International Boundary',
              'Battle Creek at International Boundary','Peoples Creek at Mouth',
              'Frenchman River at International Boundary', 'Beaver Creek Bowdoin','Rock Creek at Mouth']


In [17]:
results= pd.DataFrame(index=gauge_names)

In [18]:
# Populate the 'Cal KGE' column with calibrate_kge array
results['Cal KGE'] = calibration_kge

# Populate the 'Val KGE' column with validate_kge array
results['Val KGE'] = validation_kge

results['Cal Bias'] = calibration_bias

results['Val Bias'] = validation_bias



In [19]:
results

,Cal KGE,Val KGE,Cal Bias,Val Bias
Swiftcurrent Creek at Sherburne Reservoir,0.423353,0.388288,-19.621335,29.810010
"St. Mary River near Babb, MT",0.781725,0.831549,-15.923470,-10.658094
St. Mary River at International Boundary,0.727076,0.502351,1.393006,12.990075
Milk River at Western Crossing of International Boundary,0.338670,0.257729,-52.758947,-42.301043
North Fork Milk River above St Mary Canal near Browning,0.405020,0.455682,11.275448,11.273301
Milk River at Eastern Crossing,0.380175,0.136200,-6.079829,59.959033
Big Sandy Creek at Mouth,-0.838190,-1.798620,130.524930,209.945956
Clear Creek at Mouth,0.617100,0.485108,-6.301419,-7.298439
Lodge Creek at International Boundary,0.281174,0.309244,-60.216948,-57.989265
Battle Creek at International Boundary,0.462599,0.335490,-19.583724,27.492195
